# Creating a ML Model for the movies database

In deze oefening gaan we verder met de dataset met gegevens over verschillende films die je gebruikt hebt in de vorige oefening.

Let hierbij zeker op de volgende zaken:
* Los onderstaande vragen op door code te schrijven in de code cellen 
* Vergeet niet te antwoorden op de gestelde vragen
* Zorg ervoor dat de code vanboven naar beneden in het document uitgevoerd
* Laat de output staan van je code

Evaluatiecriteria:
* Correctheid code: 40%
* Codekwaliteit: 20%
* Tekstuele verklaring antwoorden: 40%

## Opstellen van de dataset

Hieronder krijg je eerst de code voor het downloaden en opstellen van de dataset.
Dit is de voorbeeldoplossing van de vorige oefening.
Voer deze code uit om de dataset te downloaden en in te laden voor je aan de volgende stappen begint.

In [1]:
import opendatasets as od
import pandas as pd
import numpy as np

od.download("https://www.kaggle.com/datasets/utkarshx27/movies-dataset")
df_movies = pd.read_csv("./movies-dataset/movie_dataset.csv")

df_movies_released = df_movies[df_movies.status == "Released"]

# stap 1/2/3
df_origin_with_budget = df_movies_released[df_movies_released.budget > 0]
df = df_origin_with_budget[["budget", "revenue", "vote_average"]].copy()

# stap 3
df["winst"] = (df.revenue - df.budget) / df.budget

# stap 4
df["language_ord_enc"] = pd.factorize(df_origin_with_budget.original_language)[0]

# stap 5
one_hots = df_origin_with_budget['genres'].str.get_dummies(sep=' ')
df = pd.concat([df, one_hots], axis=1)
df["Science Fiction"] = df.Science
df.drop(["Science", "Fiction"], axis=1, inplace=True)

# stap 6
def enc_runtime(value):
    if value == 0:
        return "unknown"
    elif value <= 30:
        return "kortspeelfilm"
    else:
        return "langspeelfilm"
df["runtime_enc"] = df_origin_with_budget.runtime.apply(enc_runtime)

# stap 7a,b,c
df_origin_with_budget.tagline = df_origin_with_budget.tagline.fillna('')
df_origin_with_budget.cast = df_origin_with_budget.cast.fillna('')
df_origin_with_budget.crew = df_origin_with_budget.crew.fillna('')
df["tagline_num_words"] = df_origin_with_budget.tagline.str.split(' ').apply(len)
df["cast_size"] = df_origin_with_budget.cast.str.split(' ').apply(len)
df["crew_size"] = df_origin_with_budget.crew.str.split('},').apply(len)

#stap 8
df = df.drop(["budget", "revenue"], axis=1)

display(df)

Skipping, found downloaded files in ".\movies-dataset" (use force=True to force download)


C:\Users\jens.baetens3\AppData\Local\Temp\ipykernel_26676\230137448.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_origin_with_budget.tagline = df_origin_with_budget.tagline.fillna('')
C:\Users\jens.baetens3\AppData\Local\Temp\ipykernel_26676\230137448.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_origin_with_budget.cast = df_origin_with_budget.cast.fillna('')
C:\Users\jens.baetens3\AppData\Local\Temp\ipykernel_26676\230137448.py:39: SettingWithCopyWarning: 
A value is trying to be set on

,vote_average,winst,language_ord_enc,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,...,Romance,TV,Thriller,War,Western,Science Fiction,runtime_enc,tagline_num_words,cast_size,crew_size
0,7.2,10.763566,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,langspeelfilm,5,10,153
1,6.9,2.203333,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,langspeelfilm,9,10,32
2,6.3,2.594590,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,langspeelfilm,5,10,155
3,7.6,3.339756,0,1,0,0,0,1,0,1,...,0,0,1,0,0,0,langspeelfilm,3,10,217
4,6.1,0.092843,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,langspeelfilm,7,11,132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4791,2.0,-1.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,langspeelfilm,5,9,4
4792,7.4,3.950000,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,langspeelfilm,3,10,2
4796,6.9,59.680000,0,0,0,0,0,0,0,1,...,0,0,1,0,0,1,langspeelfilm,6,10,6
4798,6.6,8.276909,4,1,0,0,0,1,0,0,...,0,0,1,0,0,0,langspeelfilm,12,11,11


## Voorspellen van de winst

Maak nu een pijplijn om de winst van een film te voorspellen.
In de oplossing moeten zeker de volgende stappen aanwezig zijn:
* Splits de dataset in features en target
* Maak een preprocessor stap aan in de pijplijn die ervoor zorgt dat alle inputs numeriek zijn. Let ook dat de correcte kolommen geschaald zijn.
* Gebruik een functie waaraan je de features (training en test), labels (training en test), de preprocessor, de regressor en een lijst van parameters aan meegeeft. Deze functie stelt de pipeline samen uit de preprocessor en de regressor, traint het model, zoekt naar de beste parameters en evalueert het model (door een aantal metrieken uit te printen)
* Zorg ervoor dat je de functie gebruikt om de modelling technieken gezien in de les te evalueren (Lineair Regression, SVM, Decision Tree, Random Forest, Nearest Neighbours).

Let hierbij zeker op de best practices voor het trainen van een model die gezien zijn in de les.

Antwoord ook zeker op de volgende vragen:
* Heb je bij 1 van de bekomen modellen gemerkt dat er overfitting/underfitting aanwezig is? Hoe heb je dit gedetecteerd? Wat heb je gedaan om dit tegen te gaan?
* In de vorige oefening is er gevraagd naar de sterkste verbanden. Hoe groot is het verschil als je enkel de 2 inputs met het sterkste verband gebruikt?
* Heb je de vote_average kolom gebruikt in je features? Is dit een vorm van data leakage? Waarom (niet)?

**Antwoord:**

In [3]:
#df.info()

In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.svm import SVR

# splits in features en targets
targets = df.winst
#features = df.drop("winst", axis=1).drop('vote_average', axis=1)
features = df[df.columns[2:]]

# splits in train en test
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2)

# preprocessor
numeric_features = ['language_ord_enc', 'tagline_num_words', 'cast_size', 'crew_size']
categoric_features = ['runtime_enc']

# geen imputers nodig want alles is niet null (zie info hierboven)
# dan moet je voor de verschillende stappen geen pipelines maken maar kan je het rechtstreeks in de columntransformer doen
preprocessor = ColumnTransformer(
    transformers= [
        ("numerics", StandardScaler(), numeric_features),
        ('categoric', OneHotEncoder(handle_unknown='ignore'), categoric_features)
    ], remainder='passthrough'
)

def search_best_model(X_train, X_test, y_train, y_test, preprocessor, regressor, parameters):

    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', regressor)
    ])

    modified_grid = {'regressor__' + key: value for key, value in parameters.items()}
    model_search = GridSearchCV(model, modified_grid, cv=5)

    model_search.fit(X_train, y_train)

    y_pred = model_search.predict(X_test)

    print('Model results:')
    print("MAE", mean_absolute_error(y_test, y_pred))
    print("MSE", mean_squared_error(y_test, y_pred))
    print("R2Score", r2_score(y_test, y_pred))

param_grid = {
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
    'C': [0.2, 0.6, 1.0, 2.0]
}

search_best_model(X_train, X_test, y_train, y_test, preprocessor, SVR(), param_grid)

Model results:
MAE 3.2851907565549876
MSE 163.48896991435473
R2Score -0.014561246320285548
